In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import sklearn

from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split



In [ ]:
input_dataset = pd.read_csv("device_db.csv", sep=",")

In [ ]:
print(input_dataset.columns)

print(input_dataset.tail())

In [ ]:
print(input_dataset["BILLING_MONTH_1"].dtype, input_dataset["BILLING_MONTH_1"].nunique())

In [ ]:
# Initialize an empty list to store column information
column_info = []

# Iterate through the columns of the DataFrame
for column in input_dataset.columns:
    column_info.append([column, input_dataset[column].dtype, input_dataset[column].nunique()])

# Create DataFrame from the list of column information
column_type_df = pd.DataFrame(column_info, columns=['Column Name', 'Column Type', 'Distinct Elements'])

print(column_type_df)

In [ ]:
# I first created an empty list to store the column statistics

column_stats = []

for columns in input_dataset.columns:
    if input_dataset[columns].dtype == 'float64':
        column_mean = input_dataset[columns].mean()
        column_median = input_dataset[columns].median()
        column_std = input_dataset[columns].std()
        column_min = input_dataset[columns].min()
        column_max = input_dataset[columns].max()

        column_stats.append([columns, column_mean, column_median, column_std, column_min, column_max])

stats_df = pd.DataFrame(column_stats, columns=['Column Name', 'Column Mean', 'Median', 'Column Standard Deviation', 'Column Minimum', 'Column Maximum'])
print(stats_df)

In [ ]:
for columns in input_dataset.columns:
    describe = input_dataset[columns].describe()
    print(describe)

In [ ]:
stats_df1 = input_dataset.describe()
stats_df1

First method
name:DEVICE_COST_MONTH_1
mean: 1.080778e+01
median: 0.000000e+00
std: 75.834602
min: 0.00
max: 6.440000e+03

DESCRIBE() method
Name: DEVICE_COST_MONTH_1, dtype: float64
count    9998.000000
mean       10.587317
std        36.995965
min         0.000000
25%         0.000000
50%         0.000000
75%         0.620000
max      1360.000000

The differences are: first the type of statistics we get. Second that there are a lot of statistical values that are different.

In [ ]:
stats_df1 = input_dataset["DEVICE_COST_MONTH_1"].describe()
stats_df1

In [ ]:
column_mean = input_dataset["DEVICE_COST_MONTH_1"].mean()
column_median = input_dataset["DEVICE_COST_MONTH_1"].median()
column_std = input_dataset["DEVICE_COST_MONTH_1"].std()
column_min = input_dataset["DEVICE_COST_MONTH_1"].min()
column_max = input_dataset["DEVICE_COST_MONTH_1"].max()
print(column_mean, column_median, column_std, column_min, column_max)


In [ ]:
previous_device_brand_census = input_dataset["PREVIOUS_DEVICE_BRAND"].value_counts()
print(previous_device_brand_census)
previous_device_model_census = input_dataset["PREVIOUS_DEVICE_MODEL"].value_counts()
print("\n",previous_device_model_census)


# 2.1

In [ ]:
nan_counter= input_dataset.isna()
nan_counter

In [ ]:
input_dataset.info()
input_dataset.dropna()

In [ ]:
clear_db = input_dataset
#for i in clear_db.columns:
#clear_db.dropna(subset=['PURCHASED_DEVICE','DEVICE_VALUE','PREVIOUS_DEVICE_MODEL'])
clear_db = clear_db.dropna(subset=['PURCHASED_DEVICE','DEVICE_VALUE','PREVIOUS_DEVICE_MODEL'])
#pd.DataFrame(clear_db)
clear_db.info()

In [ ]:
data_traffic = [col for col in clear_db.columns if ("DATA_TRAFFIC" in col) ]
voice_traffic = [col for col in clear_db.columns if ("VOICE_TRAFFIC" in col) ]
billing_month = [col for col in clear_db.columns if ("BILLING_MONTH" in col) ]
device_cost = [col for col in clear_db.columns if ("DEVICE_COST" in col) ]
columns_to_clear = data_traffic + voice_traffic+billing_month+device_cost

for col in clear_db.columns:
    if col in columns_to_clear: 
        clear_db[col] = clear_db[col].fillna(0) 
clear_db.info()

In [ ]:
print(input_dataset.shape)
clear_db.shape


# 2.2


In [ ]:
variables = ['DEVICE_VALUE', 'VOICE_TRAFFIC_MONTH_1', 'VOICE_TRAFFIC_MONTH_2', 'VOICE_TRAFFIC_MONTH_3', 'BILLING_MONTH_2', 'DURATION_LINE']

for var in variables:
    sns.histplot(data=clear_db, x=var, kde=False, alpha=0.7, label=var)

    # Adding labels and title
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title(var)
    # Adding legend
    plt.legend()
    # Showing plot
    plt.show()

In [ ]:
#we transform the variable that is exponential
clear_db['log_var'] = clear_db['VOICE_TRAFFIC_MONTH_2'].apply(lambda x: np.log(x+1))
sns.histplot(data=clear_db, x='log_var', kde=False, alpha=0.7, label='VOICE_TRAFFIC_MONTH_2')

# Adding labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('VOICE_TRAFFIC_MONTH_2')
# Adding legend
plt.legend()
# Showing plot
plt.show()

In [ ]:
variables = [ 'VOICE_TRAFFIC_MONTH_2', 'DATA_TRAFFIC_MONTH_2', 'DURATION_LINE']

for var in variables:
    clear_db[var].plot( kind = 'box')

    # Adding labels and title
    plt.ylabel('Value')
    plt.title(var)
    # Adding legend
    plt.legend()
    # Showing plot
    plt.show()

In [ ]:
data_traffic = [col for col in clear_db.columns if ("DATA_TRAFFIC" in col) ]
voice_traffic = [col for col in clear_db.columns if ("VOICE_TRAFFIC" in col) ]
columns_to_clear = data_traffic + voice_traffic
df = clear_db[columns_to_clear]
#print(columns_to_cor)
correlation_matrix = df.corr()
print(correlation_matrix)
plt.figure(figsize=(8, 6))
plt.matshow(correlation_matrix, cmap='RdBu', fignum=1)
plt.colorbar(label='Correlation')
plt.xticks(np.arange(len(columns_to_clear)), columns_to_clear, rotation=45)
plt.yticks(np.arange(len(columns_to_clear)), columns_to_clear)
plt.title('Correlation Between Traffic Attributes, Duration Line, Billing, Device Cost, and Device Value')
plt.show()

# 2.4


In [ ]:
from sklearn.preprocessing import StandardScaler

std_data_traffic_m_1 = clear_db
scaler = StandardScaler()

# Extract the column and convert it into a 2D array-like object
data_traffic_month_1 = clear_db[["DATA_TRAFFIC_MONTH_1"]]
scaler.fit(data_traffic_month_1)

# Now, you can transform the column
std_data_traffic_m_1["DATA_TRAFFIC_MONTH_1"] = scaler.transform(data_traffic_month_1)
#rint(std_data_traffic_m_1["DATA_TRAFFIC_MONTH_1"])
std_data_traffic_m_1["DATA_TRAFFIC_MONTH_1"].describe()

In [ ]:

test_df = pd.DataFrame(clear_db)
test_df["DATA_TRAFFIC_MONTH_1"] = StandardScaler().fit_transform(test_df[['DATA_TRAFFIC_MONTH_1']])
display(test_df['DATA_TRAFFIC_MONTH_1'])
test_df['DATA_TRAFFIC_MONTH_1'].describe()

In [ ]:
sns.histplot(data= test_df, x="DATA_TRAFFIC_MONTH_1", kde=False, alpha=0.7, label=var)
# Adding labels and title
plt.xlabel('DATA_TRAFFIC_MONTH_1')
plt.ylabel('Frequency')
plt.title('Histogram')
# Adding legend
plt.legend()
# Showing plot
plt.show()



In [ ]:
test_df["DATA_TRAFFIC_MONTH_1"] = MinMaxScaler().fit_transform(test_df[["DATA_TRAFFIC_MONTH_1"]])
display(test_df['DATA_TRAFFIC_MONTH_1'])
test_df["DATA_TRAFFIC_MONTH_1"].describe()
sns.histplot(data= test_df, x="DATA_TRAFFIC_MONTH_1", kde=False, alpha=0.7, label=var)
# Adding labels and title
plt.xlabel('DATA_TRAFFIC_MONTH_1')
plt.ylabel('Frequency')
plt.title('Histogram')
# Adding legend
plt.legend()
# Showing plot
plt.show()


# 2.5

In [ ]:
brand_int = pd.DataFrame(clear_db, columns = ["PREVIOUS_DEVICE_BRAND"])
brand_int["brand_encoded"] = LabelEncoder().fit_transform(brand_int['PREVIOUS_DEVICE_BRAND'])
display(brand_int)

In [ ]:
manuf_bin = pd.DataFrame(clear_db, columns = ["PREVIOUS_DEVICE_MANUF"])
manuf_bin["PREVIOUS_DEVICE_MANUF_BIN"]= LabelEncoder().fit_transform(manuf_bin['PREVIOUS_DEVICE_MANUF'])
display(manuf_bin)

# 2.7


In [ ]:

text_parsing_df = pd.DataFrame(clear_db)
text_parsing_df = test_df['PURCHASED_DEVICE'].str.split('_', expand=True)
print(text_parsing_df)



# 2.8

In [ ]:
train_df, test_df = train_test_split(clear_db, test_size=0.3, random_state=42)
train_df.info()
test_df.info()


# 3. Comparing iPhone and Samunsung J series users

I am going to study the differences between Iphone and Samsung J series users. First I will create two different dataframes, one with all the rows from iPhone users and the other one with all the rows of J series users. First I am going to decide which rows should be dropped in case of having NaN values.

<font size="1" color="gray"></font>

In [ ]:
#We upload the database
database = pd.read_csv("device_db.csv", sep=",")

If there is no data in "PURCHASED_DEVICE" or "DEVICE_VALUE" we remove those cells. If "DATA_TRAFFIC_MONTH_(1..6)", "VOICE_TRAFFIC_MONTH_(1..6)", "BILLING_MONTH_(1..6)", or "DEVICE_COST_MONTH_(1..6)" have NaN values we will consider them to be 0.
<font size="1" color="white"></font>

In [ ]:
#Now i will drop and fill the rows just explained.
data_traffic = [col for col in clear_db.columns if ("DATA_TRAFFIC" in col) ]
voice_traffic = [col for col in clear_db.columns if ("VOICE_TRAFFIC" in col) ]
billing_month = [col for col in clear_db.columns if ("BILLING_MONTH" in col) ]
device_cost = [col for col in clear_db.columns if ("DEVICE_COST" in col) ]
columns_to_clear = data_traffic + voice_traffic + billing_month + device_cost

clear_db = database

for col in database.columns:
    if col in columns_to_clear: 
        clear_db[col] = clear_db[col].fillna(0)

clear_db = clear_db.dropna(subset=['PURCHASED_DEVICE','DEVICE_VALUE'])
database = clear_db

Now that we have cleared the database I want to filter it into two databases. One for J series Samsung users and the other for iPhone users. Using the variable "PURCHASED_DEVICE" I am creating a function in order to transform the categorical value into a numerical one.
<font size="1" color="white"></font>

In [ ]:
def extract_brand(device_name):
    # Splitting the string by underscore
    parts = device_name.split('_')
    # Checking if the second part contains "Samsung" or "iPhone"
    if len(parts) > 1:
        if "SAMSUNG" in parts[1]:
            return 1
        elif "APPLE" in parts[1]:
            return 2
        else: return 0

I add a column to the dataframe with the numerical version of "PURCHASED_DEVICE".
<font size="1" color="black"></font>

In [ ]:
brand_code = []
for name in database["PURCHASED_DEVICE"]:
    #print(name)
    brand = extract_brand(str(name))
    brand_code.append(brand)

database["brand_code"] = brand_code
database["brand_code"] 


If the brand code of the row is 1 we add the row to the samsung users dataframe, and if it has a 2 to the Apple user dataframe.


In [ ]:
samsung_users =pd.DataFrame(columns=database.columns)
apple_users =pd.DataFrame(columns=database.columns)
samsung_users = database[database["brand_code"] == 1]
apple_users = database[database["brand_code"] == 2]

Now that we have samsung and apple users separated we can proceed to compare their data. I will 2 different topics to compare them between the 2 groups. I will make a device analysis using 'DEVICE_VALUE' and 'DEVICE_COST_MONTH_,  and a Traffic_analysis using DATA_TRAFFIC_MONTH_1'. I will compare their distributions and descriptive statistics and make data visualtization.
